<a href="https://colab.research.google.com/github/koh-ts/traffic_volume_prediction/blob/master/traffic_volume_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from sklearn.linear_model import Ridge
import pandas as pd
import datetime
import pandas_profiling as pdp
from IPython.display import HTML

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
root_dir = '/content/drive/My Drive/report1/'

N = 30000
nfeat = 8

train_data = pd.read_csv(root_dir + 'train.csv')
test_data = pd.read_csv(root_dir + 'test.csv')

Xmat = np.matrix(train_data)[:,:nfeat]
Ymat = np.matrix(train_data)[:,nfeat]

In [4]:
all_data = pd.concat([train_data, test_data], ignore_index=True)
all_data = all_data[['timestamp', 'holiday', 'temperature', 'weather', 'weather_detail', 'rain_in_hour', 'snow_in_hour', 'clouds_cover', 'traffic_volume']]
# all_data.head(4)
# all_data[['holiday'][:2]]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [5]:
profile1 = pdp.ProfileReport(all_data)
profile1.to_file(outputfile="myoutputfile1.html")
HTML(filename='myoutputfile1.html')

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,9
Number of observations,45645
Total Missing (%),1.9%
Total size in memory,3.1 MiB
Average record size in memory,72.0 B
Numeric,5
Categorical,4
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [0]:
# cnt = 0
for index, d in all_data.iterrows():
  # cnt += 1
  n = int(d['timestamp'][11:13])
  # print(all_data.iat[index, 1])
  if all_data.iat[index - n, 1] != 'None':
    all_data.at[index, 'isholiday'] = 1
    # print(all_data.at[index, 'isholiday'])
  else:
    all_data.at[index, 'isholiday'] = 0
    # print(all_data.at[index, 'isholiday'])
  all_data.at[index, 'time'] = n // 2
  # if 0 <= n < 4:
  #   all_data['time'] = 0
  # elif 4 <= n < 8:
  #   all_data['time'] = 1
  # elif 8 <= n < 12:
  #   all_data['time'] = 2
  # elif 12 <= n < 16:
  #   all_data['time'] = 3
  # elif 16 <= n < 20:
  #   all_data['time'] = 4
  # elif 20 <= n < 24:
  #   all_data['time'] = 5
  # if cnt == 100:
  #   break

In [0]:
all_data['temperature'] = all_data['temperature'].apply(lambda x : x * 9 / 5 - 459.67)
all_data['weather_detail'] = all_data['weather_detail'].str.replace('Sky is clear', 'clear')
all_data['weather_detail'] = all_data['weather_detail'].str.replace('sky is clear', 'clear')
all_data['weather_detail'] = all_data['weather_detail'].str.replace('Sky is Clear', 'clear')

In [0]:
meta_t = pd.get_dummies(all_data['time'])
meta_w = pd.get_dummies(all_data['weather'])
meta_wd = pd.get_dummies(all_data['weather_detail'])

In [9]:
profile2 = pdp.ProfileReport(all_data)
profile2.to_file(outputfile="myoutputfile2.html")
HTML(filename='myoutputfile2.html')

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,11
Number of observations,45645
Total Missing (%),1.6%
Total size in memory,3.8 MiB
Average record size in memory,88.0 B
Numeric,6
Categorical,4
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [0]:
del all_data['timestamp']
del all_data['weather']
del all_data['holiday']
del all_data['weather_detail']

In [0]:
# all_data = all_data[['temperature', 'rain_in_hour', 'snow_in_hour', 'clouds_cover', 'time', 'isholiday', 'traffic_volume']]
all_data = all_data[['temperature', 'rain_in_hour', 'snow_in_hour', 'clouds_cover', 'time', 'traffic_volume']]
all_data.columns
l = len(all_data.columns) - 1
# all_data.head(3)

In [0]:
# all_data.iloc[:3, :6]
# Xmat[:,1:5]
# meta_w.head(3)
# meta_wd.head(3)
# meta_t.head(3)

In [0]:
X = np.float64(np.hstack([meta_w.iloc[:len(train_data),:],meta_wd.iloc[:len(train_data),:],meta_t.iloc[:len(train_data),:],all_data.iloc[:len(train_data),:l]]))
Y = np.float64(Ymat)

XTrain = X[:N,:]
YTrain = Y[:N,:]
XVal = X[N:,:]
YVal = Y[N:,:]

In [0]:
reg = Ridge(alpha=90).fit(XTrain, YTrain)

In [15]:
def RMSE(vec1, vec2):
  return np.sqrt(np.mean(np.power(vec1-vec2,2)))

yHatTrain = reg.predict(XTrain)
yHatVal = reg.predict(XVal)
print("Training error ", RMSE(YTrain, yHatTrain))
print("Validation error ", RMSE(YVal, yHatVal))

Training error  992.1670107292783
Validation error  942.7571145605524


In [0]:
# XTest = np.float64(np.hstack([test_data.iloc[:,1:5],meta_w.iloc[len(train_data):,:],meta_wd.iloc[len(train_data):,:],all_data.iloc[len(train_data):,-1:]]))
XTest = np.float64(np.hstack([meta_w.iloc[len(train_data):,:],meta_wd.iloc[len(train_data):,:],meta_t.iloc[len(train_data):,:],all_data.iloc[len(train_data):,:l]]))
# XTest.shape
# XTrain.shape

In [0]:
yHatTest = reg.predict(XTest)

np.savetxt(root_dir+'result.txt', yHatTest) #save predictions in rows